In [24]:
import torch 
import torch.nn as nn   
import os 
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [25]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 59 
set_seed(seed)

In [26]:
root_dir = 'data/weather-dataset/dataset'
img_paths = []
labels = []
classes = {
    label_idx : class_name \
        for label_idx, class_name in enumerate(sorted(os.listdir(root_dir)))
}



for label_idx, class_name in classes.items():
    class_dir = os.path.join(root_dir, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img_paths.append(img_path)
        labels.append(label_idx)

print(classes)
print(labels[2000])
print(img_paths[2000])

{0: 'dew', 1: 'fogsmog', 2: 'frost', 3: 'glaze', 4: 'hail', 5: 'lightning', 6: 'rain', 7: 'rainbow', 8: 'rime', 9: 'sandstorm', 10: 'snow'}
2
data/weather-dataset/dataset\frost\4051.jpg


In [33]:
val_size = 0.2
test_size = 0.125
is_shuffle = True 

X_train,X_val, y_train, y_val = train_test_split(img_paths, labels, test_size= val_size, random_state= seed, shuffle = is_shuffle)
X_train,X_test, y_train, y_test = train_test_split(X_train, y_train, test_size= test_size, random_state= seed, shuffle = is_shuffle)


In [ ]:
class WeatherDataset(Dataset):
    def __init__(self, X,y,transform = None):
        self.transform = transform
        self.img_paths = X
        self.labels = y

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, self.labels[idx]
def transform(img, img_size = (224,224)):
    img = img.resize(img_size)
    img = np.array(img)[...,:3]
    img = torch.tensor(img).permute(2,0,1).float()
    normalized_img = img/255.0

    return normalized_img


(tensor([[[0.7412, 0.7412, 0.7333,  ..., 0.5961, 0.6706, 0.7176],
          [0.7176, 0.7333, 0.7333,  ..., 0.6353, 0.6706, 0.6157],
          [0.7098, 0.7294, 0.7333,  ..., 0.6627, 0.6549, 0.5765],
          ...,
          [0.6510, 0.6471, 0.6431,  ..., 0.6431, 0.6588, 0.6667],
          [0.6549, 0.6510, 0.6471,  ..., 0.6196, 0.6196, 0.6275],
          [0.6431, 0.6431, 0.6431,  ..., 0.6196, 0.6235, 0.6275]],
 
         [[0.7922, 0.8000, 0.7882,  ..., 0.6039, 0.6784, 0.7255],
          [0.7765, 0.7882, 0.7843,  ..., 0.6392, 0.6706, 0.6157],
          [0.7647, 0.7843, 0.7882,  ..., 0.6667, 0.6588, 0.5804],
          ...,
          [0.6902, 0.6863, 0.6824,  ..., 0.6902, 0.7059, 0.7137],
          [0.6941, 0.6902, 0.6863,  ..., 0.6667, 0.6667, 0.6745],
          [0.6824, 0.6824, 0.6824,  ..., 0.6667, 0.6706, 0.6745]],
 
         [[0.8235, 0.8275, 0.8196,  ..., 0.5882, 0.6667, 0.7137],
          [0.8039, 0.8196, 0.8235,  ..., 0.6235, 0.6627, 0.6078],
          [0.8000, 0.8196, 0.8275,  ...,

In [ ]:
#Dataset for train test val

train_dataset = WeatherDataset(X_train, y_train, transform= transform)
val_dataset = WeatherDataset(X_val,y_val, transform=transform)
test_dataset = WeatherDataset(X_test, y_test, transform= transform)


In [36]:
# Dataloader 

train_batch_size = 512
test_batch_size = 8 

train_loader = DataLoader(
    train_dataset,
    batch_size= train_batch_size,
    shuffle= True
)

test_loader = DataLoader(
    test_dataset,
    batch_size= test_batch_size,
    shuffle= False
)

val_loader = DataLoader(
    val_dataset,
    batch_size= test_batch_size,
    shuffle= False
)

In [46]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride= stride, padding= 1)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride= 1, padding= 1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size= 1,stride= stride),
                nn.BatchNorm2d(out_channels)
            )
        self.relu = nn.ReLU()

    def forward(self, x):
        shortcut = x.clone()
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x += self.downsample(shortcut)
        x = self.relu(x)

        return x

In [47]:
class ResNet(nn.Module):
    def __init__(self, residual_block, n_block_lst, n_classes):
        super(ResNet,self).__init__()
        self.conv1 = nn.Conv2d(3,64,kernel_size= 7, stride= 2, padding= 3)
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride= 2, padding= 1)
        self.conv2 = self.create_layer(residual_block, 64, 64, n_block_lst[0], 1)
        self.conv3 = self.create_layer(residual_block, 64, 128,n_block_lst[1],2)
        self.conv4 = self.create_layer(residual_block, 128,256,n_block_lst[2], 2)
        self.conv5 = self.create_layer(residual_block, 256,512,n_block_lst[3], 2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, n_classes)
    def create_layer(self, residual_block, in_channels, out_channels, n_blocks, stride):
        blocks = []
        first_block = residual_block(in_channels, out_channels, stride)
        blocks.append(first_block)

        for idx in range(1, n_blocks):
            block = residual_block(out_channels, out_channels, stride= 1)
            blocks.append(block)
        block_sequential = nn.Sequential(*blocks)
        return block_sequential
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.maxpool(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x= self.avgpool(x)
        x = self.flatten(x)
        x = self.fc1(x)

        return x

In [ ]:
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ResNet(
    residual_block= ResidualBlock,
    n_block_lst= [2,2,2,2],
    n_classes= n_classes
).to(device)

# print(torch.cuda.is_available())

False


In [ ]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    losses = []
    with torch.no_grad():
        for inputs , labels in dataloader :
            inputs , labels = inputs .to( device ) , labels.to( device )
            outputs = model(inputs)
            loss = criterion (outputs, labels)
            losses.append(loss.item())
            _, predicted = torch.max (outputs.data, 1)
            total += labels.size (0)
            correct += (predicted == labels).sum().item ()

    loss = sum(losses) / len (losses)
    acc = correct / total

    return loss, acc


In [51]:
def fit(model, train_loader,val_loader,criterion, optimizer, device, epochs):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        batch_train_losses= []
        model.train()
        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            batch_train_losses.append(loss.item())

        train_loss = sum(batch_train_losses) / len(batch_train_losses)

        train_losses.append(train_loss)

        val_loss, val_acc = evaluate(model ,val_loader, criterion,device)
        val_losses.append(val_loss)

        print(f'EPOCH { epoch + 1}:\tTrain loss :{train_loss:.4f}\tVal loss : {val_loss:.4f}')
        return train_losses, val_losses

In [52]:
lr = 1e-2
epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = lr)



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
train_losses, val_losses = fit(model, train_loader, val_loader, criterion, optimizer, device, epochs)


In [ ]:
val_loss, val_acc = evaluate(
    model,
    val_loader,
    criterion,
    device
)

test_loss, test_acc = evaluate(
    model,
    test_loader,
    criterion,
    device
)

print('evaluation on val/test dataset')
print('Val accuracy: ',val_acc)
print('Test accuracy: ', test_acc)